In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Aditya\\OneDrive\\Documents\\DataScience\\projects\\text-summarizer'

##### Configuration

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir : Path
    source_url : Path
    local_data_file_path : Path
    unzip_dir_path : Path

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

#### Configuration Updates

In [6]:
class ConfigurationManager:
    def __init__(self,
            config_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file_path=config.local_data_file_path,
            unzip_dir_path=config.unzip_dir_path
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logging

#### Data Ingestion Component

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file_path):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file_path
            )
            logging.info(f"File is downloaded")

        else:
            logging.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip file path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path=self.config.unzip_dir_path
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file_path,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2024-12-09 23:07:50,745 : INFO : common : yaml file: config\config.yaml loaded successfully ]
[2024-12-09 23:07:50,749 : INFO : common : yaml file: params.yaml loaded successfully ]
[2024-12-09 23:07:50,753 : INFO : common : Created directory at : artifacts ]
[2024-12-09 23:07:50,757 : INFO : common : Created directory at : artifacts/data_ingestion ]
[2024-12-09 23:07:51,176 : INFO : 2219660700 : File is downloaded ]
